In [1]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'
# a_pattern = 'SI06122.Chn'

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Esta é a nova classe recomendada
from sklearn import linear_model

import pandas as pd
from random import randrange
# import plotly.graph_objects as go

In [3]:
from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [4]:
ogra = Ograyspy()
print(ogra.info_node)
to_be_found = 'Genie_Transfer'
# to_be_found = 'some_spectra' # NAO ESTAH ACHANDO localmente no note!

Found folder name:  C:\Users\marce\.anaconda
Parts:  ('C:\\', 'Users', 'marce', '.anaconda')
No. spec files:  0
Pedro-notebook


In [5]:
print('\nExec ogra.define_files_folder(to_be_found):')
ogra.define_files_folder(to_be_found)
print(ogra.pkl_folder_files)
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
del ogra


Exec ogra.define_files_folder(to_be_found):
Found folder name:  C:\Users\marce\OwnDrive\Genie_Transfer
Parts:  ('C:\\', 'Users', 'marce', 'OwnDrive', 'Genie_Transfer')
No. spec files:  7316
WindowsPedro-notebook.pkl


In [6]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [7]:
len(reduc_nms)

7316

In [8]:
# 2022-Dez-28 Esse é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06122.Chn'

In [9]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=7212, nomearq = Si/SI2022/SI06122.Chn
Final choices:
spectra_path: C:\Users\marce\OwnDrive\Genie_Transfer
a_spec_name: C:\Users\marce\OwnDrive\Genie_Transfer\Si\SI2022\SI06122.Chn
reduced_f_name: Si/SI2022/SI06122.Chn


(WindowsPath('C:/Users/marce/OwnDrive/Genie_Transfer/Si/SI2022/SI06122.Chn'),
 'Si/SI2022/SI06122.Chn')

In [10]:
nucl_iear1_df = pd.read_pickle('nucl_iear1_list.pkl')
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma
1,24na,1368.6250,0.005,99.994,0.002,14.956,B-,100.0,1368.6,True
2,24na,2754.0080,0.011,99.867,0.01,14.956,B-,100.0,1368.6,False
9,51cr,320.0824,0.0004,9.910,0.01,27.704,EC,100.0,320.1,True
15,54mn,834.8480,0.003,99.976,0.001,312.200,EC+B+,100.0,834.8,True
22,58co,810.7593,0.002,99.450,0.01,70.860,EC+B+,100.0,810.8,True
...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,,2.010,0.19,6.150,B-,100.0,911.6,False
1067,228Ac,93.3470,,3.200,0.3,6.150,B-,100.0,911.6,False
1068,228Ac,105.5660,,1.160,0.11,6.150,B-,100.0,911.6,False
1069,228Ac,106.8940,,1.540,0.15,6.150,B-,100.0,911.6,False


In [11]:
# 2023-Jun-15: Setting gamma lines/ranges to dismiss in the analysis
df1 = nucl_iear1_df
df1["is_to_consider"] = True
df1.loc[((df1.energy > 509) & (df1.energy < 513)) | (df1.energy < 100), "is_to_consider"] = False
# Convert these columns from object to numeric; empty strings turns into NaNs
df1["unc_i"] = pd.to_numeric(df1.unc_i, errors="coerce")
df1["unc_en"] = pd.to_numeric(df1.unc_en, errors="coerce")
del df1
nucl_iear1_df

,nuclide_name,energy,unc_en,intensity,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider
1,24na,1368.6250,0.0050,99.994,0.002,14.956,B-,100.0,1368.6,True,True
2,24na,2754.0080,0.0110,99.867,0.010,14.956,B-,100.0,1368.6,False,True
9,51cr,320.0824,0.0004,9.910,0.010,27.704,EC,100.0,320.1,True,True
15,54mn,834.8480,0.0030,99.976,0.001,312.200,EC+B+,100.0,834.8,True,True
22,58co,810.7593,0.0020,99.450,0.010,70.860,EC+B+,100.0,810.8,True,True
...,...,...,...,...,...,...,...,...,...,...,...
1066,228Ac,89.9540,NaN,2.010,0.190,6.150,B-,100.0,911.6,False,False
1067,228Ac,93.3470,NaN,3.200,0.300,6.150,B-,100.0,911.6,False,False
1068,228Ac,105.5660,NaN,1.160,0.110,6.150,B-,100.0,911.6,False,True
1069,228Ac,106.8940,NaN,1.540,0.150,6.150,B-,100.0,911.6,False,True


In [12]:
a_spec = Spec(f_name, reduced_f_name)

In [13]:
dir(a_spec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_engy_to_pks_df',
 'channel_energy_calib',
 'create_activities_df',
 'curr_h_win',
 'det_descr',
 'en_eff',
 'f_name',
 'final_composed_baseline',
 'generate_pandas_dataframe',
 'identify_nuclides',
 'lv_time',
 'net_spec_ser_an',
 'nucl_an',
 'origin_spec_ser_an',
 'read_pkl_file',
 'reduced_f_name',
 'results_pkl_file',
 'rl_time',
 'sam_descr',
 'source_datetime',
 'spec_io',
 'start_datetime',
 'sufx',
 'total_analysis']

In [14]:
a_spec.total_analysis(gener_dataframe=True, results_path='../ograyspy_results')
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

Starting Spec.total_analysis...
resolve_peaks_and_regions:
define_multiplets_regions completado. Define: self.mix_regions.
resolve_peaks_and_regions:
define_multiplets_regions completado. Define: self.mix_regions.
Si\SI2022\SI06122.Chn
Si\SI2022\SI06122_result.pkl
results_pkl_file
..\ograyspy_results\Si\SI2022\SI06122_result.pkl
Finish Spec.total_analysis!


{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x2a4186fc750>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x2a41860ce90>,
 'f_name': WindowsPath('C:/Users/marce/OwnDrive/Genie_Transfer/Si/SI2022/SI06122.Chn'),
 'reduced_f_name': 'Si/SI2022/SI06122.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.96,
 'source_datetime': None,
 'results_pkl_file': WindowsPath('../ograyspy_results/Si/SI2022/SI06122_result.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x2a41860f2d0>,
 'start_datetime': datetime.datetime(2022, 7, 27, 16, 56, 29),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-061/22 Agua da Piscina Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x2a416d53990>,
 'nucl_an': <nuclide_analysis_class.NuclideAnalysis at 0x2a416d52d90>}

In [15]:
a_spec.identify_nuclides(nucl_iear1_df)

Lets identify nuclides.
    peaks   fwhm_centr   rough_sums    centroids     variances       pk_hei  \
0      30    29.785567  1893.942368    29.963695  14133.214332   818.646195   
1     128   128.364887  2876.788495   127.767831  11283.211505   720.585627   
2     145   144.459911  1278.607977   144.896569  10161.329960   389.571554   
3     236   235.273124   462.759320   235.202199  12019.150830   123.425784   
4     396   396.401754   630.310441   396.426778  10404.641509   162.845302   
5     489   488.997063   249.976688   488.967375   4072.023312   125.089174   
6     581   581.463638   430.396173   581.647407   5405.821287   147.276132   
7     716   715.508125   222.738094   715.666805   4033.829966    79.006167   
8     841   841.967813  2900.206925   841.976062   7873.573309   559.895371   
9     960   960.088168   483.773366   960.065118   3335.841582   148.624963   
10   1003  1002.888165   769.886444  1002.931019   3458.113556   220.739274   
11   1011  1011.595555   246

In [16]:
aux_3, pks_comprehensive_df, df_result, d5_grouped, res_1 = (
    a_spec.create_activities_df(nucl_iear1_df, results_path='../ograyspy_results')
)

In [17]:
aux_3

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,unc_i,half_life,decay,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,recalib_engy_ransac,new_delta_en
355,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,...,0.070000,3.053000e+00,B-,100.000,583.2,False,False,1.110802,74.757552,-0.212448
362,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,...,0.177555,1.062200e+01,B-,100.000,238.6,False,False,1.265802,74.757619,-0.057381
386,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,...,0.169913,2.706000e+01,B-,100.000,351.9,False,False,1.265802,74.757619,-0.057381
591,145,144.459911,1278.607977,144.896569,10161.329960,389.571554,4.276963,247.018556,389.571554,142,...,0.030000,3.053000e+00,B-,100.000,583.2,False,False,1.561152,85.219578,0.233578
1065,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,...,0.530718,1.062200e+01,B-,100.000,238.6,True,True,1.610260,238.848356,0.216356
1319,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,...,0.112495,2.706000e+01,B-,100.000,351.9,False,True,1.564299,295.369964,0.145964
1555,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,...,0.235274,2.706000e+01,B-,100.000,351.9,True,True,1.487539,351.976722,0.044722
1887,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,...,NaN,7.086000e+01,EC+B+,100.000,810.8,False,False,1.490852,510.979365,-0.020635
1891,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,...,NaN,2.439300e+02,EC+B+,100.000,1115.5,False,False,1.490852,510.979365,-0.020635
1993,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,...,0.200000,3.053000e+00,B-,100.000,583.2,False,False,1.720852,510.979464,0.209464


In [18]:
pks_comprehensive_df

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,rg_bas,widths,wi_hei,lf_ips,rg_ips,engy_pk_det,recalib_energy
0,30,29.785567,1893.942368,29.963695,14133.214332,818.646195,201.465549,450.836680,818.646195,28,33,2.244704,409.323097,28.663214,30.907919,16.318529,14.983281
1,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,137,2.761022,360.292813,126.984376,129.745398,76.080802,74.726202
2,145,144.459911,1278.607977,144.896569,10161.329960,389.571554,4.276963,247.018556,389.571554,142,151,2.657272,194.785777,143.131275,145.788547,86.547152,85.189121
3,236,235.273124,462.759320,235.202199,12019.150830,123.425784,40.769738,84.205044,123.425784,233,240,2.919528,61.712892,233.813360,236.732888,141.727537,140.351208
4,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,404,3.333009,81.422651,394.735249,398.068258,240.242260,238.832399
5,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,493,1.411704,62.544587,488.291211,489.702915,296.788299,295.358690
6,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,585,2.564867,73.638066,580.181204,582.746071,353.419539,351.969786
7,716,715.508125,222.738094,715.666805,4033.829966,79.006167,15.639623,46.454617,79.006167,713,719,2.684467,39.503084,714.165892,716.850359,435.310799,433.831268
8,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,850,4.639239,279.947685,839.648194,844.287433,512.490852,510.982554
9,960,960.088168,483.773366,960.065118,3335.841582,148.624963,51.372759,40.677023,148.624963,955,963,2.878815,74.312482,958.648761,961.527576,584.648031,583.112222


In [20]:
df_result

,peaks,fwhm_centr,rough_sums,centroids,variances,pk_hei,lf_thr,rg_thr,promns,lf_bas,...,decay_%,key_gamma,is_key_gamma,is_to_consider,delta_en,disintegr,activity_conc,acti_conc_unc,one_s2,x_s2
355,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,...,100.000,583.2,False,False,-0.243798,1.418964e+06,283.792807,0.212612,2.212190e+01,6.278037e+03
362,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,...,100.000,238.6,False,False,-0.088798,4.876457e+05,97.529143,0.063767,2.459317e+02,2.398551e+04
386,128,128.364887,2876.788495,127.767831,11283.211505,720.585627,359.256221,71.539300,720.585627,100,...,100.000,351.9,False,False,-0.088798,9.365137e+05,187.302735,0.225063,1.974203e+01,3.697737e+03
591,145,144.459911,1278.607977,144.896569,10161.329960,389.571554,4.276963,247.018556,389.571554,142,...,100.000,583.2,False,False,0.203121,1.995901e+06,399.180219,0.800103,1.562096e+00,6.235579e+02
1065,396,396.401754,630.310441,396.426778,10404.641509,162.845302,70.909899,49.239266,162.845302,392,...,100.000,238.6,True,True,0.200399,7.324243e+04,14.648486,0.028856,1.200997e+03,1.759279e+04
1319,489,488.997063,249.976688,488.967375,4072.023312,125.089174,88.241458,88.978919,125.089174,485,...,100.000,351.9,False,True,0.134690,8.502770e+04,17.005540,0.026439,1.430588e+03,2.432792e+04
1555,581,581.463638,430.396173,581.647407,5405.821287,147.276132,89.934593,1.069452,147.276132,579,...,100.000,351.9,True,True,0.037786,8.929519e+04,17.859037,0.020093,2.476997e+03,4.423678e+04
1887,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,...,100.000,810.8,False,False,-0.017446,9.783745e+05,195.674894,NaN,NaN,NaN
1891,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,...,100.000,1115.5,False,False,-0.017446,1.025925e+07,2051.850244,NaN,NaN,NaN
1993,841,841.967813,2900.206925,841.976062,7873.573309,559.895371,213.926360,11.088422,559.895371,836,...,100.000,583.2,False,False,0.212554,1.289698e+06,257.939655,0.069839,2.050250e+02,5.288407e+04


In [21]:
df_result.columns

Index(['peaks', 'fwhm_centr', 'rough_sums', 'centroids', 'variances', 'pk_hei',
       'lf_thr', 'rg_thr', 'promns', 'lf_bas', 'rg_bas', 'widths', 'wi_hei',
       'lf_ips', 'rg_ips', 'engy_pk_det', 'recalib_energy', 'nuclide_name',
       'energy', 'unc_en', 'intensity', 'unc_i', 'half_life', 'decay',
       'decay_%', 'key_gamma', 'is_key_gamma', 'is_to_consider', 'delta_en',
       'disintegr', 'activity_conc', 'acti_conc_unc', 'one_s2', 'x_s2'],
      dtype='object')

In [22]:
df_result[['peaks','rough_sums','nuclide_name']]

,peaks,rough_sums,nuclide_name
355,128,2876.788495,208Tl
362,128,2876.788495,212Pb
386,128,2876.788495,214Pb
591,145,1278.607977,208Tl
1065,396,630.310441,212Pb
1319,489,249.976688,214Pb
1555,581,430.396173,214Pb
1887,841,2900.206925,58co
1891,841,2900.206925,65zn
1993,841,2900.206925,208Tl


In [23]:
d5_grouped

In [24]:
res_1

,one_s2,x_s2,avrg_xm,avrg_sd
nuclide_name,,,,
110ag,4.024029e+05,2.134244e+07,53.037505,0.001576
131i,1.141399e+00,2.629009e+02,230.332119,0.936011
137cs,8.920315e+04,1.008007e+06,11.300127,0.003348
208Tl,3.592875e+04,5.081915e+05,14.144426,0.005276
212Pb,1.446929e+03,4.157830e+04,28.735552,0.026289
214Bi,7.893428e+03,3.856648e+05,48.858982,0.011256
214Pb,3.927327e+03,7.226243e+04,18.399904,0.015957
228Ac,1.538554e+02,1.042419e+04,67.753175,0.080620
24na,2.799598e+09,1.828842e+10,6.532518,0.000019


In [25]:
# 2023-Oct-25 PAREI AQUI. Reformatar-deletar tudo o que segue.

In [ ]:
justtostopthenotebook

In [ ]:
spectra_list_df

In [ ]:
nucl_iear1_selctd_gamms_df = nucl_iear1_df.loc [
    (nucl_iear1_df.intensity > 1.0) & nucl_iear1_df.is_to_consider
]
nucl_iear1_selctd_gamms_df

In [ ]:
nucl_iear1_selctd_gamms_df.dtypes

In [ ]:
# Primeiro fiz assim:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
# cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
# cross_df

# Agora fazer de novo:
# cross_df_2 = pd.merge(matching_peaks_df, nucl_iear1_selctd_gamms_df, how='cross', suffixes=('_x', None))
# cross_df_2.drop(cross_df_2.filter(regex='_x$').columns, axis=1, inplace=True)
# cross_df_2["delta_en"] = cross_df_2.recalib_engy_ransac - cross_df_2.energy
# cross_df_2

In [ ]:
# cross_df = pd.merge(pks_comprehensive_df, nucl_iear1_selctd_gamms_df, how='cross')
 #    cross_df["delta_en"] = cross_df.engy_pk_det - cross_df.energy
  #  cross_df 

In [ ]:
# 2023-Oct-25
# Proceed from here.

In [ ]:
new_ch_en_calib

In [ ]:
def en_eff(engy):
    return np.exp(p_eff(np.log(engy)))

In [ ]:
en_eff(1332)

In [ ]:
# 2023-Sep-21
# Now use the assembled peak df, read_result_df, to proceed.

In [ ]:
# 2023-Oct-24: FIX
# Integrate what follows here

In [ ]:
analysis_df.reset_index(drop=True, inplace=True)

In [ ]:
analysis_df['test'] = analysis_df['decay_%']*analysis_df['intensity']/100

In [ ]:
analysis_df

In [ ]:
analysis_df.reindex(['nuclide_name'])

In [ ]:
# d5_where = d5.where(d5.nuclide_name=="110ag")
# d5_where
d5_loc = analysis_df.loc[analysis_df.nuclide_name=="110ag"]
d5_loc

In [ ]:
# 2023-Ago-4 PAREI AQUI. Fazer a indexação para o cálculo da atividade considerando a média das linhas de cada nuclídeo
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#dataframe-column-selection-in-groupby
d5_grouped = analysis_df.groupby("nuclide_name")[['energy','intensity','centroids','rough_sums','variances','disintegr']]
# d5_grouped = d5.groupby("nuclide_name")[['disintegr']]
# sdfsdf = pd.DataFrame([[d5_grouped.mean(), d5_grouped.median()]]) 
# sdfsdf


In [ ]:
d5_grouped.mean()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#built-in-aggregation-methods
d5_aggr_df = pd.DataFrame(df_result.groupby("nuclide_name")[['disintegr']].mean())
d5_aggr_df['dsnt_median'] = df_result.groupby("nuclide_name")[['disintegr']].median()
d5_aggr_df['dsnt_counts'] = df_result.groupby("nuclide_name")[['disintegr']].count()
d5_aggr_df['dsnt_sem'] = df_result.groupby("nuclide_name")[['disintegr']].sem()
d5_aggr_df['dsnt_std'] = df_result.groupby("nuclide_name")[['disintegr']].std()
d5_aggr_df

In [ ]:
df_result.groupby("nuclide_name")[['disintegr']].describe()

In [ ]:
d5_aggr_df['disint_mean'] = d5_grouped.mean()
d5_aggr_df['disint_count'] = d5_grouped.count()
d5_aggr_df

In [ ]:
df_result

In [ ]:
help(spec_engy_identif)

In [ ]:
# 2023-Jun-13
# PAREI AQUI
# Trazer o trabalho feito em
# radionuclide_analysis.ipynb e adaptar ao que está aqui
# especialmente na identificaćão dos nucl[ideos

# Depois, deletar radionuclide_analysis.ipynb.

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)


df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)


frames = [df1, df2, df3]

result = pd.concat(frames)
result

In [ ]:
result = pd.concat(frames, keys=["x", "y", "z"])
result

In [ ]:
result.loc[result.A=='A1'].loc[result.C=='C0']

In [ ]:
# https://stackoverflow.com/questions/63529555

In [ ]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [ ]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

In [ ]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

In [ ]:
# 2023-Oct-24 FIX:
# Brought from extinct radionuculde_analysis.ipynb
# Jupyter notebook for tests with OGRaySpY modules

In [ ]:
pip list

In [ ]:
import numpy as np
from numpy.random import default_rng
from numpy.polynomial import Polynomial as P
from scipy import special
import pandas as pd

from ograyspy_class import select_spectrum_from_folder_list
from spec_class import Spec

In [ ]:
# https://lmfit.github.io/lmfit-py/builtin_models.html
from lmfit import Model, minimize, Parameters, report_fit, fit_report, printfuncs
from lmfit.models import LinearModel, StepModel, ExponentialModel, LorentzianModel
from lmfit.models import GaussianModel, SkewedGaussianModel, ExpressionModel

In [ ]:
from ograyspy_class import Ograyspy

In [ ]:
ogra = Ograyspy(batch_mode=False, folder_to_find='Genie_Transfer')
print('Objeto ogra: Ograyspy criado.')
print(f'ogra.info_plat: {ogra.info_plat}')
print(f'ogra.info_mach: {ogra.info_mach}')
print(f'ogra.info_syst: {ogra.info_syst}')
print(f'ogra.info_node: {ogra.info_node}')
print(f'ogra.home_path: {ogra.home_path}')

In [ ]:
# a spectrum pattern to find and test:

In [ ]:
# 2022
# Se quiser ignorar o espectro aleatório, definir aqui um específico. Se não, pular.
# Vou ignorar o espectro já analisado e refazer a análise, para testes.
# 2022-Mar-24 Resolver essa questão dos nomes de arquivos em Linux e Windows
# a_spec_name = '..\\..\\Genie_Transfer\\Si\\2018\\PNR\\PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/PNR/PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/CTP/CTP0908-2.Chn'
# Esse é um que dá problema na baseline perto do 159 keV do I-123:
# a_spec_name = '../../Genie_Transfer/Si/2018/CCI/CCI1508I.Chn'
# Precisa quebrar mais a baseline: Ok, consertado!!!
# 2022: Já usando o GMX:
# a_spec_name = '../../Genie_Transfer/Filtros/2022/Cci/CCI1603-I.Chn'



In [ ]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [ ]:
dir(ogra)

In [ ]:
vars(ogra)

In [ ]:
# Select a machine/system to read from:
spectra_list_df = pd.read_pickle(ogra.pkl_folder_files)
spectra_list_df

In [ ]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [ ]:
reduc_nms

In [ ]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

In [ ]:
a_spec_name, reduced_f_name

In [ ]:
a_spec = Spec(a_spec_name, reduced_f_name)

In [ ]:
vars(a_spec)

In [ ]:
# 2023-Jun-13
# PAREI AQUI: levar tudo que está aqui para
# nuclide_identification.ipynb


In [ ]:
p_en = a_spec.channel_energy_calib.p_en
p_en

In [ ]:
peaks_df['energy'] = p_en(peaks_df['centroids'])
peaks_df

In [ ]:
# 2023-Mar-21: Identify nuclide candidates in peaks_df from pisciro_eng_df, with energy tolerance en_toler

In [ ]:
big_merge = pd.merge(peaks_df, pisciro_eng_df, how='cross')
big_merge

In [ ]:
en_toler = 2.0

In [ ]:
big_merge['delta_en'] = big_merge.energy - big_merge.Engy
big_merge

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-callable
# MaTched ENergies
mt_en = pd.DataFrame(big_merge.loc[lambda df: abs(df['delta_en']) < en_toler, :]) \
[['Engy', 'Nucl', 'rough_sums', 'Yie']]
mt_en
# mt_en[['Engy', 'Nucl']]

In [ ]:

msedg serg set asrhg saertrg

In [ ]:
rng = np.random.default_rng()
# rints = rng.integers(low=0, high=10, size=3)

# 2022-Jun-23: Reativar após consertar o arquivo pickle:
# rints = rng.integers(pkl_df1.count()[0], size=3)
# print(rints)
# type(rints[0])
# some_spec = rints[1]
# pkl_df1.iloc[some_spec]
# a_spec_name = pkl_df1.iloc[some_spec]['FName']
# a_spec_name

In [ ]:
# 2022-Dez-28: Atualizar:
# (2022-Jun-24 Lembrando como estão os defaults de total_analysis(...):
# def total_analysis(self, k_sep_pk=2.0, smoo=3000.0, widths_range=(4.0, 20.0)):)
# smoo=3000 fica razoável, mas faz umas ondas
# smoo=1500 fica horrível, faz uns abismos. Quanto menor, pior
# smoo=400 fica razoável, mas faz umas ondas
# ogra.perform_total_analysis(
#     k_sep_pk=2.0, smoo=4096, widths_range=(4.0, 20.0)
# )

In [ ]:
# 2022-Jun-14
# Verificando o que se tem até agora em a_raw_spec:
# (Pode pular e ir direto aos gráficos)

In [ ]:

# Testes:
# https://numpy.org/doc/stable/reference/random/generator.html#distributions
# x = np.linspace(0, 10, N) + rng.normal(loc=7, scale=0.05, size=N)
# y = np.linspace(22, 23, N)

# Busca do canal onde a altura do espectro se "estabiliza"
# https://numpy.org/doc/stable/reference/routines.sort.html#searching



In [ ]:
vars(pkprms)

In [ ]:
# 2022-Dez-21 Achar o pico de maior área inicial do espectro:

In [ ]:
largest_pk = np.argmax(pkprms.rough_sums)
pkprms.centroids[largest_pk]

In [ ]:
# Só pra lembrar como faz divisão inteira
21 // 7

In [ ]:
wi_re = pkprms.wide_regions[largest_pk]
x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
y = cntarr[wi_re[0]:wi_re[1]+1]
c1 =  pkprms.centroids[largest_pk]
s1 = pkprms.propts['widths'][largest_pk]*pkprms.k_fwhm_to_sd
a1 = pkprms.rough_sums[largest_pk]
# c2 = 313
# s2 = 1.5
# a2 = 400
sste = 2.0

In [ ]:
# 2023-Jan-3
# Aqui finalizo este teste:
# Dado um espectro, escolhi o pico mais proeminente do espectro
# e fiz o ajuste gaussianco com lmfit
# No caso, o pico tem área, centroide, SD e FWHM:
# g1_amplitude:  5264.84728 +/- 85.1465566 (1.62%) (init = 5347.497)
# g1_center:     1926.29564 +/- 0.02690572 (0.00%) (init = 1926.275)
# g1_sigma:      1.44040103 +/- 0.02690060 (1.87%) (init = 1.418733)
# g1_fwhm:       3.39188516 +/- 0.06334608 (1.87%) == '2.3548200*g1_sigma'
# g1_height:     1458.18438 +/- 23.5824677 (1.62%) == '0.3989423*g1_amplitude/max(1e-15, g1_sigma)'

# Comparando com saíuda do Genie:
# canal (>0)    En:    FWHM (keV)    net area    net area unc   continuum
# 1927.34      1173.47 2.04          5.40E+003   89.73          8.83E+002
#

In [ ]:
out

In [ ]:
out.params

In [ ]:
out.var_names

In [ ]:
out.params.items()

In [ ]:
out.params.pretty_print()

In [ ]:
print('-------------------------------')
print('Parameter    Value       Stderr')
for name, param in out.params.items():
    # print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')
    print(name)
    print(param)
    print(param.value)

In [ ]:
def gauss_lmfit_singlet(pkprms, ipk):
    wi_re = pkprms.wide_regions[ipk]
    print(f'pico: {wi_re}')
    x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
    y = cntarr[wi_re[0]:wi_re[1]+1]
    c1 = pkprms.centroids[ipk]
    s1 = pkprms.propts['widths'][ipk]*pkprms.k_fwhm_to_sd
    a1 = pkprms.rough_sums[ipk]

    parames = Parameters()

    gauss1 = GaussianModel(prefix='g1_')
    parames.update(gauss1.make_params())
    parames['g1_center'].set(value=c1, min=c1-2, max=c1+2)
    parames['g1_sigma'].set(value=s1, min=0.5, max=10)
    parames['g1_amplitude'].set(value=a1, min=10)

    mod = gauss1

    init = mod.eval(parames, x=x)
    out = mod.fit(y, parames, x=x, verbose=False)
    print('Fez.')

    # ret = [param for param in out.params.items()]
    ret = out
    return ret

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    print(ret)

In [ ]:
ret

In [ ]:
ret.params

In [ ]:
for name, param in ret.params.items():
    print(name, param.value, param.stderr)

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    # ret.params.pretty_print()

In [ ]:
ret

In [ ]:
xfgn sr hsrthf srth

In [ ]:
my_df.dtypes

In [ ]:
ch_en = ogra.a_spec.channel_energy_calib.p_en

In [ ]:
my_df['centrs_en'] = ch_en(my_df['centroids'])
my_df

In [ ]:
# The End